In [1]:
from collections import defaultdict, Counter, OrderedDict
import numpy as np

In [2]:
f = open("inputs/aoc7.txt").read()
hands = [s.split(" ") for s in f.splitlines()]
hands = [(i[0], int(i[1])) for i in hands]

In [3]:
hands[0:5]

[('8T64Q', 595), ('79J27', 258), ('88885', 88), ('8933J', 444), ('72527', 676)]

In [4]:
#
points = OrderedDict({
    'five' : 1000000e5,
    'four' : 100000e5,
    'full_house':10000e5,
    'three' : 1000e5,
    'two_pair': 100e5,
    'one_pair' : 10e5,
    'high' : 1e5})


# 1

In [5]:
cards = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3","2"]

def assign_string(hand, cards_order):
    abc = 'abcdefghijklmnopqrstuvwxyz'
    new_string = ""
    for card in hand:
        new_string=new_string+abc[cards_order.index(card)]       
    return new_string

In [6]:
assign_string("88885", cards)

'ggggj'

In [7]:
def hand_type(hand):
    
    c=Counter()
    for card in hand:
        c[card]+=1
    
    # five
    if len(c)==1:
        ht = "five"
    
    any_four = any([cnt==4 for cnt in c.values()])
    if any_four:
        ht="four"
        
    any_three = any([cnt==3 for cnt in c.values()])
    if any_three and len(c)==2:
        ht= "full_house"
    
    if any_three and len(c)>2:
        ht="three"
    
    any_two = any([cnt==2 for cnt in c.values()])
    if any_two and len(c)==3:
        ht="two_pair"
        
    if any_two and len(c)==4:
        ht="one_pair"
    if len(c)==5:
        ht="high" 
    return ht

In [8]:
def compute_sum(hands, cards_order):
    
    # first order hands by by handtype
    groups = defaultdict(list)
    for hand_data in hands:
        #print(hand)
        ht = hand_type(hand_data[0])
        groups[ht].append(hand_data)

    # assign strings that allow ordering of equal handtypes
    new_groups = defaultdict(list)
    for ht, hands in groups.items():
        new_strings = [(hand_data[0],assign_string(hand_data[0], 
                                                   cards_order=cards_order),hand_data[1]) for hand_data in hands]
        new_strings.sort(key=lambda x:x[1])
        new_groups[ht]=new_strings
    
    # order by stringv
    new_order = []
    for k in points:
        new_order.extend(new_groups[k])
    new_order.reverse()
    
    # sum up
    som = 0
    for idx,data in enumerate(new_order):
        rank = idx+1
        som = som + data[2]*rank
    return som

In [9]:
compute_sum(hands, cards_order=cards)

250474325

# 2

In [10]:
new_cards_order = ["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3","2","J"]

In [11]:
from itertools import product

In [12]:
def swap_characters(hand, tuple, indices):
    for idx,position in enumerate(indices):
        hand = list(hand)
        hand[position]=tuple[idx]
        hand="".join(hand)
    return hand

In [13]:
swap_characters("ABCDE", ("Z","X"),[3,1])

'AXCZE'

In [14]:
def generate_hands(hand):
    
    num_J = hand.count("J")
    perms = list(product(cards,repeat=num_J)) 
    new_hands = []
    positions_with_J=[]
    for idx, card in enumerate(hand):
        if card=="J":
            positions_with_J.append(idx)
    
    for combi in perms:
        new_hand = swap_characters(hand, tuple=combi, indices=positions_with_J)
        new_hands.append(new_hand)

    return new_hands

In [18]:
from copy import copy

def get_highest_hand_type(hand):
    
    best_hand = hand
    best_ht = hand_type(hand)
    best_points = points[best_ht]
    
    if "J" in hand:
        new_hands = generate_hands(hand)
        for new_hand in new_hands:
            
            ht_new_hand = hand_type(new_hand)
            points_new_hand = points[ht_new_hand]
            if points_new_hand>best_points:
                best_hand = new_hand
                best_ht = ht_new_hand
                best_points = points_new_hand

            else:
                continue
    return best_ht
                    

In [19]:
def compute_sum(hands, cards_order):
    
    # first order hands by by handtype
    groups = defaultdict(list)
    for hand_data in hands:
        #print(hand)
        ht = get_highest_hand_type(hand_data[0])
        groups[ht].append(hand_data)

    # assign strings that allow ordering of equal handtypes
    new_groups = defaultdict(list)
    for ht, hands in groups.items():
        new_strings = [(hand_data[0],assign_string(hand_data[0], 
                                                   cards_order=cards_order),hand_data[1]) for hand_data in hands]
        new_strings.sort(key=lambda x:x[1])
        new_groups[ht]=new_strings
    
    # order by stringv
    new_order = []
    for k in points:
        new_order.extend(new_groups[k])
    new_order.reverse()
    
    # sum up
    som = 0
    for idx,data in enumerate(new_order):
        rank = idx+1
        som = som + data[2]*rank
    return som

In [20]:
compute_sum(hands, cards_order=new_cards_order)

248909434